In [14]:

#mount drive
from google.colab import drive
drive.mount('/content/MyDrive')
import seaborn as sns
sns.set_theme("paper")



Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [15]:
# @title Initialize Config

import torch
import numpy
class Config:
    def __init__(self, **kwargs):
        self.channels_imu_acc = kwargs.get('channels_imu_acc', [])
        self.channels_imu_gyr = kwargs.get('channels_imu_gyr', [])
        self.channels_joints = kwargs.get('channels_joints', [])
        self.channels_emg = kwargs.get('channels_emg', [])
        self.seed = kwargs.get('seed', 42)
        self.data_folder_name = kwargs.get('data_folder_name', 'default_data_folder_name')
        self.dataset_root = kwargs.get('dataset_root', 'default_dataset_root')
        self.imu_transforms = kwargs.get('imu_transforms', [])
        self.joint_transforms = kwargs.get('joint_transforms', [])
        self.emg_transforms = kwargs.get('emg_transforms', [])
        self.input_format = kwargs.get('input_format', 'csv')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = Config(
    data_folder_name='/content/MyDrive/MyDrive/sd_datacollection_v4/all_subjects_data.h5',
    dataset_root='/content/datasets',
    input_format="csv",
    channels_imu_acc=['ACCX1', 'ACCY1', 'ACCZ1','ACCX2', 'ACCY2', 'ACCZ2', 'ACCX3', 'ACCY3', 'ACCZ3', 'ACCX4', 'ACCY4', 'ACCZ4', 'ACCX5', 'ACCY5', 'ACCZ5', 'ACCX6', 'ACCY6', 'ACCZ6'],
    channels_imu_gyr=['GYROX1', 'GYROY1', 'GYROZ1', 'GYROX2', 'GYROY2', 'GYROZ2', 'GYROX3', 'GYROY3', 'GYROZ3', 'GYROX4', 'GYROY4', 'GYROZ4', 'GYROX5', 'GYROY5', 'GYROZ5', 'GYROX6', 'GYROY6', 'GYROZ6'],
    channels_joints=['elbow_flex_r', 'arm_flex_r', 'arm_add_r'],
    channels_emg=['IM EMG4', 'IM EMG5', 'IM EMG6'],
)

#set seeds
torch.manual_seed(config.seed)
numpy.random.seed(config.seed)


In [16]:
class DataSharder:
    def __init__(self, config, split):
        self.config = config
        self.h5_file_path = config.data_folder_name  # Path to the HDF5 file
        self.split = split

    def load_data(self, subjects, window_length, window_overlap, dataset_name):
        print(f"Processing subjects: {subjects} with window length: {window_length}, overlap: {window_overlap}")

        self.window_length = window_length
        self.window_overlap = window_overlap

        # Process the data from the HDF5 file
        self._process_and_save_patients_h5(subjects, dataset_name)

    def _process_and_save_patients_h5(self, subjects, dataset_name):
        # Open the HDF5 file
        with h5py.File(self.h5_file_path, 'r') as h5_file:
            dataset_folder = os.path.join(self.config.dataset_root, dataset_name, self.split).replace("subject", "").replace("__", "_")
            print("Dataset folder:", dataset_folder)

            if os.path.exists(dataset_folder):
                print("Dataset Exists, Skipping...")
                return

            os.makedirs(dataset_folder, exist_ok=True)
            print("Dataset folder created: ", dataset_folder)

            for subject_id in tqdm(subjects, desc="Processing subjects"):
                subject_key = subject_id
                if subject_key not in h5_file:
                    print(f"Subject {subject_key} not found in the HDF5 file. Skipping.")
                    continue

                subject_data = h5_file[subject_key]
                session_keys = list(subject_data.keys())  # Sessions for this subject

                for session_id in session_keys:
                    session_data_group = subject_data[session_id]

                    for sessions_speed in session_data_group.keys():
                        session_data = session_data_group[sessions_speed]

                        # Extract IMU, EMG, and Joint data as numpy arrays
                        imu_data, imu_columns = self._extract_channel_data(session_data, self.config.channels_imu_acc + self.config.channels_imu_gyr)
                        emg_data, emg_columns = self._extract_channel_data(session_data, self.config.channels_emg)
                        joint_data, joint_columns = self._extract_channel_data(session_data, self.config.channels_joints)

                        # Shard the data into windows and save each window
                        self._save_windowed_data(imu_data, emg_data, joint_data, subject_key, session_id,sessions_speed, dataset_folder, imu_columns, emg_columns, joint_columns)

    def _save_windowed_data(self, imu_data, emg_data, joint_data, subject_key, session_id, session_speed, dataset_folder, imu_columns, emg_columns, joint_columns):
        window_size = self.window_length
        overlap = self.window_overlap
        step_size = window_size - overlap

        # Path to the CSV log file
        csv_file_path = os.path.join(dataset_folder, '..', f"{self.split}_info.csv")

        # Ensure the folder exists
        os.makedirs(dataset_folder, exist_ok=True)

        # Prepare CSV log headers (ensure the columns are 'file_name' and 'file_path')
        csv_headers = ['file_name', 'file_path']

        # Create or append to the CSV log file
        file_exists = os.path.isfile(csv_file_path)
        with open(csv_file_path, mode='a', newline='') as csv_file:
            writer = csv.writer(csv_file)

            # Write the headers only if the file is new
            if not file_exists:
                writer.writerow(csv_headers)

            # Determine the total data length based on the minimum length across the data sources
            total_data_length = min(imu_data.shape[1], emg_data.shape[1], joint_data.shape[1])

            # Adjust the starting point for windows based on total data length
            start = 2000 if total_data_length > 4000 else 0

            # Ensure that each window across imu_data, emg_data, and joint_data has the same shape before concatenation
            for i in range(start, total_data_length - window_size + 1, step_size):
                imu_window = imu_data[:, i:i + window_size]
                emg_window = emg_data[:, i:i + window_size]
                joint_window = joint_data[:, i:i + window_size]

                # Check if the window sizes are valid
                if imu_window.shape[1] == window_size and emg_window.shape[1] == window_size and joint_window.shape[1] == window_size:
                    # Convert windowed data to pandas DataFrame



                    imu_df = pd.DataFrame(imu_window.T, columns=imu_columns)
                    emg_df = pd.DataFrame(emg_window.T, columns=emg_columns)
                    joint_df = pd.DataFrame(joint_window.T, columns=joint_columns)



                    # Concatenate the data along the column axis
                    combined_df = pd.concat([imu_df, emg_df, joint_df], axis=1)

                    # Save the combined windowed data as a CSV file
                    file_name = f"{subject_key}_{session_id}_{session_speed}_win_{i}_ws{window_size}_ol{overlap}.csv"
                    file_path = os.path.join(dataset_folder, file_name)
                    combined_df.to_csv(file_path, index=False)

                    # Log the file name and path in the CSV (in the correct columns)
                    writer.writerow([file_name, file_path])
                else:
                    print(f"Skipping window {i} due to mismatched window sizes.")

    def _extract_channel_data(self, session_data, channels):
        """
        Extracts data for the given channels from the dataset (whether it's a compound dataset or simple dataset),
        and interpolates missing values (NaNs) in each channel data.
        """
        extracted_data = []
        column_names = []

        if isinstance(session_data, h5py.Dataset):
            # Check if the dataset has named fields (compound dataset)
            if session_data.dtype.names:
                # Compound dataset, use the named fields
                column_names = session_data.dtype.names
                for channel in channels:
                    if channel in column_names:
                        channel_data = session_data[channel][:]  # Access by field name
                        # Convert the data to a numeric type (float), if necessary
                        channel_data = pd.to_numeric(channel_data, errors='coerce')
                        # Interpolate NaN values
                        df = pd.DataFrame(channel_data)
                        df_interpolated = df.interpolate(method='linear', axis=0, limit_direction='both')
                        extracted_data.append(df_interpolated.to_numpy().flatten())
                    else:
                        print(f"Channel {channel} not found in compound dataset.")
            else:
                # Simple dataset, use index-based access (no named fields)
                column_names = session_data.attrs.get('column_names', [])

                # Cast column_names to a list to allow 'index' lookup
                column_names = list(column_names)
                new_column_names = []

                assert len(column_names) > 0, "column_names not found in dataset attributes"
                for channel in channels:
                    if channel in column_names:
                        col_idx = column_names.index(channel)
                        new_column_names.append(channel)
                        channel_data = session_data[:, col_idx]  # Access by column index

                        # Convert the data to a numeric type (float), if necessary
                        channel_data = pd.to_numeric(channel_data, errors='coerce')

                        # Interpolate NaN values
                        df = pd.DataFrame(channel_data)
                        df_interpolated = df.interpolate(method='linear', axis=0, limit_direction='both')
                        extracted_data.append(df_interpolated.to_numpy().flatten())
                    else:
                        print(f"Channel {channel} not found in session data.")

        return np.array(extracted_data), new_column_names

In [17]:
# @title Dataset creation
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from torch.utils.data import ConcatDataset
import random
from torch.utils.data import TensorDataset

class ImuJointPairDataset(Dataset):
    def __init__(self, config, subjects, window_length, window_overlap, split='train', dataset_train_name='train', dataset_test_name='test'):
        self.config = config
        self.split = split
        self.subjects = subjects
        self.window_length = window_length
        self.window_overlap = window_overlap if split == 'train' else 0
        self.input_format = config.input_format
        self.channels_imu_acc = config.channels_imu_acc
        self.channels_imu_gyr = config.channels_imu_gyr
        self.channels_joints = config.channels_joints
        self.channels_emg = config.channels_emg

        # Convert the list of subjects to a string that is path-safe
        subjects_str = "_".join(map(str, subjects)).replace('subject', '').replace('__', '_')

        # Use dataset_train_name or dataset_test_name based on split
        if split == 'train':
            dataset_name = f"dataset_wl{self.window_length}_ol{self.window_overlap}_train{subjects_str}"
        else:
            dataset_name = f"dataset_wl{self.window_length}_ol{self.window_overlap}_test{subjects_str}"

        self.dataset_name = dataset_name

        # Define the root directory based on dataset name
        self.root_dir = os.path.join(self.config.dataset_root, self.dataset_name)

        # Ensure sharded data exists, if not, reshard
        self.ensure_resharded(subjects, dataset_train_name if split == 'train' else dataset_test_name)

        info_path = os.path.join(self.root_dir, f"{split}_info.csv")
        self.data = pd.read_csv(info_path)

    def ensure_resharded(self, subjects, dataset_name):
        if not os.path.exists(self.root_dir):
            print(f"Sharded data not found at {self.root_dir}. Resharding...")
            data_sharder = DataSharder(self.config,self.split)
            # Pass dynamic parameters to sharder
            data_sharder.load_data(subjects, window_length=self.window_length, window_overlap=self.window_overlap, dataset_name=self.dataset_name)
        else:
            print(f"Sharded data found at {self.root_dir}. Skipping resharding.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path = os.path.join(self.root_dir,self.split, self.data.iloc[idx, 0])

        if self.input_format == "csv":
            combined_data = pd.read_csv(file_path)
        else:
            raise ValueError("Unsupported input format: {}".format(self.input_format))

        imu_data_acc, imu_data_gyr, joint_data, emg_data = self._extract_and_transform(combined_data)
        return imu_data_acc, imu_data_gyr, joint_data, emg_data

    def _extract_and_transform(self, combined_data):
        imu_data_acc = self._extract_channels(combined_data, self.channels_imu_acc)
        imu_data_gyr = self._extract_channels(combined_data, self.channels_imu_gyr)
        joint_data = self._extract_channels(combined_data, self.channels_joints)
        emg_data = self._extract_channels(combined_data, self.channels_emg)

        imu_data_acc = self.apply_transforms(imu_data_acc, self.config.imu_transforms)
        imu_data_gyr = self.apply_transforms(imu_data_gyr, self.config.imu_transforms)
        joint_data = self.apply_transforms(joint_data, self.config.joint_transforms)
        emg_data = self.apply_transforms(emg_data, self.config.emg_transforms)

        return imu_data_acc, imu_data_gyr, joint_data, emg_data

    def _extract_channels(self, combined_data, channels):
        return combined_data[channels].values if self.input_format == "csv" else combined_data[:, channels]

    def apply_transforms(self, data, transforms):
        for transform in transforms:
            data = transform(data)
        return torch.tensor(data, dtype=torch.float32)

class ImuJointPairSubjectDataset(ImuJointPairDataset):
    def __init__(self, config, subjects, window_length, window_overlap, split='train', dataset_train_name='train', dataset_test_name='test'):
        super().__init__(config, subjects, window_length, window_overlap, split, dataset_train_name, dataset_test_name)

        # Create a mapping from subject strings (e.g., 'subject_1') to class indices
        self.subject_mapping = {subject: i for i, subject in enumerate(sorted(subjects))}

    def __getitem__(self, idx):
        # Retrieve the original data from the parent class
        imu_data_acc, imu_data_gyr, joint_data, emg_data = super().__getitem__(idx)

        # Get the filename from the data index
        filename = self.data.iloc[idx, 0]

        # Extract the subject ID from the filename
        filename_base = os.path.basename(filename)
        filename_without_ext = os.path.splitext(filename_base)[0]
        parts = filename_without_ext.split('_')

        # Construct subject string in the format 'subject_x'
        try:
            subject_index = parts.index('subject')
            subject_str = f"subject_{parts[subject_index + 1]}"  # Create string like 'subject_1'
        except ValueError:
            raise ValueError(f"'subject' not found in filename: {filename}")
        except IndexError:
            raise ValueError(f"Subject ID not found after 'subject' in filename: {filename}")

        # Map subject_str to class index
        if subject_str not in self.subject_mapping:
            raise ValueError(f"Subject ID {subject_str} not found in training set.")

        mapped_class = self.subject_mapping[subject_str]

        # Return class index instead of one-hot encoding
        return imu_data_acc, imu_data_gyr, joint_data, emg_data, mapped_class


def create_base_data_loaders(
    config,
    train_subjects,
    test_subjects,
    window_length=100,
    window_overlap=75,
    batch_size=64,
    dataset_train_name='train',
    dataset_test_name='test'
):
    # Create datasets with explicit parameters
    train_dataset = ImuJointPairSubjectDataset(
        config=config,
        subjects=train_subjects,
        window_length=window_length,
        window_overlap=window_overlap,
        split='train',
        dataset_train_name=dataset_train_name
    )

    test_dataset = ImuJointPairSubjectDataset(
        config=config,
        subjects=test_subjects,
        window_length=window_length,
        window_overlap=0,  # Typically no overlap in test set
        split='test',
        dataset_test_name=dataset_test_name
    )

    # Split train dataset into training and validation sets
    train_size = int(0.9 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader




In [18]:
# @title Kinematicsnet Architecture
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from scipy.signal import butter, filtfilt
from sklearn.metrics import mean_squared_error
import numpy as np
class Encoder_1(nn.Module):
    def __init__(self, input_dim, dropout):
        super(Encoder_1, self).__init__()
        self.lstm_1 = nn.LSTM(input_dim, 128, bidirectional=True, batch_first=True, dropout=0)
        self.lstm_2 = nn.LSTM(256, 64, bidirectional=True, batch_first=True, dropout=0)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128, 32)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x):
        out_1, (h_1, _) = self.lstm_1(x)
        out_1 = self.dropout_1(out_1)
        out_2, (h_2, _) = self.lstm_2(out_1)
        out_2 = self.dropout_2(out_2)
        return out_2, (h_1, h_2)

class Encoder_2(nn.Module):
    def __init__(self, input_dim, dropout):
        super(Encoder_2, self).__init__()
        self.gru_1 = nn.GRU(input_dim, 128, bidirectional=True, batch_first=True, dropout=0)
        self.gru_2 = nn.GRU(256, 64, bidirectional=True, batch_first=True, dropout=0)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128, 32)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x):
        out_1, h_1 = self.gru_1(x)
        out_1 = self.dropout_1(out_1)
        out_2, h_2 = self.gru_2(out_1)
        out_2 = self.dropout_2(out_2)
        return out_2, (h_1, h_2)


class GatingModule(nn.Module):
    def __init__(self, input_size):
        super(GatingModule, self).__init__()
        self.gate = nn.Sequential(
            nn.Linear(2*input_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, input1, input2):
        # Apply gating mechanism
        gate_output = self.gate(torch.cat((input1,input2),dim=-1))

        # Scale the inputs based on the gate output
        gated_input1 = input1 * gate_output
        gated_input2 = input2 * (1 - gate_output)

        # Combine the gated inputs
        output = gated_input1 + gated_input2
        return output
#variable w needs to be checked for correct value, stand-in value used


from torch.autograd import Function

class GradientReversalFunction(Function):
    @staticmethod
    def forward(ctx, x, lambda_grl):
        ctx.lambda_grl = lambda_grl
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        lambda_grl = ctx.lambda_grl
        grad_input = grad_output.neg() * lambda_grl
        return grad_input, None

def GradientReversalLayer(lambda_grl):
    return GradientReversalFunction.apply

class teacher(nn.Module):
    def __init__(self, input_acc, input_gyr, input_emg, drop_prob=0.25, w=100,num_subjects=12,lambda_grl=1.0):
        super(teacher, self).__init__()

        self.lambda_grl = lambda_grl
        self.w=w
        self.encoder_1_acc=Encoder_1(input_acc, drop_prob)
        self.encoder_1_gyr=Encoder_1(input_gyr, drop_prob)
        self.encoder_1_emg=Encoder_1(input_emg, drop_prob)

        self.encoder_2_acc=Encoder_2(input_acc, drop_prob)
        self.encoder_2_gyr=Encoder_2(input_gyr, drop_prob)
        self.encoder_2_emg=Encoder_2(input_emg, drop_prob)

        self.BN_acc= nn.BatchNorm1d(input_acc, affine=False)
        self.BN_gyr= nn.BatchNorm1d(input_gyr, affine=False)
        self.BN_emg= nn.BatchNorm1d(input_emg, affine=False)


        self.fc = nn.Linear(2*3*128+128,3)
        self.fc_classifier = nn.Linear(2*3*128+128,num_subjects)
        self.dropout=nn.Dropout(p=.05)

        self.gate_1=GatingModule(128)
        self.gate_2=GatingModule(128)
        self.gate_3=GatingModule(128)

        self.fc_kd = nn.Linear(3*128, 2*128)

               # Define the gating network
        self.weighted_feat = nn.Sequential(
            nn.Linear(128, 1),
            nn.Sigmoid())

        self.attention=nn.MultiheadAttention(3*128,4,batch_first=True)
        self.gating_net = nn.Sequential(nn.Linear(128*3, 3*128), nn.Sigmoid())
        self.gating_net_1 = nn.Sequential(nn.Linear(2*3*128+128, 2*3*128+128), nn.Sigmoid())

        self.pool = nn.MaxPool1d(kernel_size=2)


    def forward(self, x_acc, x_gyr, x_emg):

        x_acc_1=x_acc.view(x_acc.size(0)*x_acc.size(1),x_acc.size(-1))
        x_gyr_1=x_gyr.view(x_gyr.size(0)*x_gyr.size(1),x_gyr.size(-1))
        x_emg_1=x_emg.view(x_emg.size(0)*x_emg.size(1),x_emg.size(-1))

        x_acc_1=self.BN_acc(x_acc_1)
        x_gyr_1=self.BN_gyr(x_gyr_1)
        x_emg_1=self.BN_emg(x_emg_1)

        x_acc_2=x_acc_1.view(-1, self.w, x_acc_1.size(-1))
        x_gyr_2=x_gyr_1.view(-1, self.w, x_gyr_1.size(-1))
        x_emg_2=x_emg_1.view(-1, self.w, x_emg_1.size(-1))

        # Pass through Encoder 1 for each modality and capture hidden states
        x_acc_1, (h_acc_1, _) = self.encoder_1_acc(x_acc_2)
        x_gyr_1, (h_gyr_1, _) = self.encoder_1_gyr(x_gyr_2)
        x_emg_1, (h_emg_1, _) = self.encoder_1_emg(x_emg_2)

        # Pass through Encoder 2 for each modality and capture hidden states
        x_acc_2, (h_acc_2, _) = self.encoder_2_acc(x_acc_2)
        x_gyr_2, (h_gyr_2, _) = self.encoder_2_gyr(x_gyr_2)
        x_emg_2, (h_emg_2, _) = self.encoder_2_emg(x_emg_2)

        # x_acc=torch.cat((x_acc_1,x_acc_2),dim=-1)
        # x_gyr=torch.cat((x_gyr_1,x_gyr_2),dim=-1)
        # x_emg=torch.cat((x_emg_1,x_emg_2),dim=-1)

        x_acc=self.gate_1(x_acc_1,x_acc_2)
        x_gyr=self.gate_2(x_gyr_1,x_gyr_2)
        x_emg=self.gate_3(x_emg_1,x_emg_2)

        x=torch.cat((x_acc,x_gyr,x_emg),dim=-1)
        x_kd=self.fc_kd(x)


        out_1, attn_output_weights=self.attention(x,x,x)

        gating_weights = self.gating_net(x)
        out_2=gating_weights*x

        weights_1 = self.weighted_feat(x[:,:,0:128])
        weights_2 = self.weighted_feat(x[:,:,128:2*128])
        weights_3 = self.weighted_feat(x[:,:,2*128:3*128])
        x_1=weights_1*x[:,:,0:128]
        x_2=weights_2*x[:,:,128:2*128]
        x_3=weights_3*x[:,:,2*128:3*128]
        out_3=x_1+x_2+x_3

        out=torch.cat((out_1,out_2,out_3),dim=-1)

        gating_weights_1 = self.gating_net_1(out)
        out=gating_weights_1*out

        #print shape of out
        out_task=self.fc(out)

        grl = GradientReversalLayer(self.lambda_grl)
        out_rev = grl(out, self.lambda_grl)  # Apply GRL to reverse gradients

        # Domain classification with reversed gradients
        out_rev = out_rev.permute(0, 2, 1)  # Permute to [batch_size, features, time]
        out_rev = torch.nn.functional.adaptive_avg_pool1d(out_rev, 1)  # Pooling
        out_rev = out_rev.squeeze(-1)  # Shape: [batch_size, features]

        out_classifier = self.fc_classifier(out_rev)  # Domain classifier output


        #print(out.shape)
        return out_task, out_classifier, (h_acc_1, h_acc_2, h_gyr_1, h_gyr_2, h_emg_1, h_emg_2)




In [19]:
# @title Loss Functions
import statistics

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
    def forward(self, output, target):
        loss = torch.sqrt(torch.mean((output - target) ** 2))
        return loss

#prediction function
def RMSE_prediction(yhat_4,test_y, output_dim,print_losses=True):

  s1=yhat_4.shape[0]*yhat_4.shape[1]

  test_o=test_y.reshape((s1,output_dim))
  yhat=yhat_4.reshape((s1,output_dim))




  y_1_no=yhat[:,0]
  y_2_no=yhat[:,1]
  y_3_no=yhat[:,2]

  y_1=y_1_no
  y_2=y_2_no
  y_3=y_3_no


  y_test_1=test_o[:,0]
  y_test_2=test_o[:,1]
  y_test_3=test_o[:,2]



  cutoff=6
  fs=200
  order=4

  nyq = 0.5 * fs
  ## filtering data ##
  def butter_lowpass_filter(data, cutoff, fs, order):
      normal_cutoff = cutoff / nyq
      # Get the filter coefficients
      b, a = butter(order, normal_cutoff, btype='low', analog=False)
      y = filtfilt(b, a, data)
      return y



  Z_1=y_1
  Z_2=y_2
  Z_3=y_3



  ###calculate RMSE

  rmse_1 =((np.sqrt(mean_squared_error(y_test_1,y_1))))
  rmse_2 =((np.sqrt(mean_squared_error(y_test_2,y_2))))
  rmse_3 =((np.sqrt(mean_squared_error(y_test_3,y_3))))





  p_1=np.corrcoef(y_1, y_test_1)[0, 1]
  p_2=np.corrcoef(y_2, y_test_2)[0, 1]
  p_3=np.corrcoef(y_3, y_test_3)[0, 1]




              ### Correlation ###
  p=np.array([p_1,p_2,p_3])
  #,p_4,p_5,p_6,p_7])




      #### Mean and standard deviation ####

  rmse=np.array([rmse_1,rmse_2,rmse_3])
  #,rmse_4,rmse_5,rmse_6,rmse_7])

      #### Mean and standard deviation ####
  m=statistics.mean(rmse)
  SD=statistics.stdev(rmse)


  m_c=statistics.mean(p)
  SD_c=statistics.stdev(p)


  if print_losses:
    print(rmse_1)
    print(rmse_2)
    print(rmse_3)
    print("\n")
    print(p_1)
    print(p_2)
    print(p_3)
    print('Mean: %.3f' % m,'+/- %.3f' %SD)
    print('Mean: %.3f' % m_c,'+/- %.3f' %SD_c)

  return rmse, p, Z_1,Z_2,Z_3
  #,Z_4,Z_5,Z_6,Z_7

def compute_biomechanical_loss(predicted_angles):
    """
    Compute the biomechanical loss (L_bio) to enforce joint limits
    for the three joints, using index-based access.
    """
    # Define joint limits for each channel (in degrees)
    min_limits = torch.tensor([0, -90, -120], device=predicted_angles.device)
    max_limits = torch.tensor([150, 180, 90], device=predicted_angles.device)

    while min_limits.dim() < predicted_angles.dim():
        min_limits = min_limits.unsqueeze(0)
    while max_limits.dim() < predicted_angles.dim():
        max_limits = max_limits.unsqueeze(0)

    # Now min_limits and max_limits have shape [1, 1, num_joints] and will broadcast correctly
    lower_violation = torch.relu(min_limits - predicted_angles)
    upper_violation = torch.relu(predicted_angles - max_limits)

    L_bio = torch.mean(lower_violation + upper_violation)

    return L_bio


class BoundRmseLoss(nn.Module):
    def __init__(self, lambda_bio=0.1):
        super(BoundRmseLoss, self).__init__()
        self.lambda_bio = lambda_bio
        self.rmse_loss = RMSELoss()  # Using your existing RMSELoss class

    def forward(self, output, target):
        # Compute RMSE loss
        L_data = self.rmse_loss(output, target)

        # Compute biomechanical loss
        L_bio = compute_biomechanical_loss(output)

        # Total loss
        total_loss = L_data + self.lambda_bio * L_bio

        return total_loss

In [20]:
# @title Model Utils

# Evaluation function
def evaluate_model(device, model, loader, criterion):
    """Runs evaluation on the validation or test set."""
    model.eval()
    total_loss = 0.0
    total_pcc = np.zeros(len(config.channels_joints))
    total_rmse = np.zeros(len(config.channels_joints))

    with torch.no_grad():
        for i, (data_acc, data_gyr, target, data_EMG,_) in enumerate(loader):
            output= model(data_acc.to(device).float(), data_gyr.to(device).float(), data_EMG.to(device).float())

            if isinstance(model, teacher):
                output,knowledge_distillation,_ = output
                loss = criterion(output, target.to(device).float())
            else:
                loss = criterion(output, target.to(device).float())

            batch_rmse, batch_pcc, _, _, _ = RMSE_prediction(output.detach().cpu().numpy(), target.detach().cpu().numpy(), len(config.channels_joints), print_losses=False)
            total_loss += loss.item()
            total_pcc += batch_pcc
            total_rmse += batch_rmse

    avg_loss = total_loss / len(loader)
    avg_pcc = total_pcc / len(loader)
    avg_rmse = total_rmse / len(loader)

    return avg_loss, avg_pcc, avg_rmse



def save_checkpoint(model, optimizer, epoch, filename, train_loss, val_loss, test_loss=None,
                    channelwise_metrics=None, history=None, curriculum_schedule=None):

    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'train_channelwise_metrics': channelwise_metrics['train'],
        'val_channelwise_metrics': channelwise_metrics['val'],
    }

    if test_loss is not None:
        checkpoint['test_loss'] = test_loss
        checkpoint['test_channelwise_metrics'] = channelwise_metrics['test']

    # Save the history (losses, PCCs, RMSEs, channel-wise metrics)
    if history:
        checkpoint['history'] = history

    # Save curriculum schedule
    if curriculum_schedule:
        checkpoint['curriculum_schedule'] = curriculum_schedule

    torch.save(checkpoint, filename)
    print(f"Checkpoint saved for epoch {epoch + 1}")



def train_teacher(
    device,
    train_loader,
    val_loader,
    test_loader,
    learn_rate,
    epochs,
    model,
    filename,
    loss_function,
    optimizer=None,
    l1_lambda=None,
    train_from_last_epoch=False,
    curriculum_loader=None,
    num_classes=12,  # Add num_classes parameter
    alpha=1.0,       # Weighting factor for classification loss
):
    model.to(device)
    criterion_regression = loss_function
    criterion_classification = nn.CrossEntropyLoss(ignore_index=-1)

    if optimizer is None:
        # Create a default Adam optimizer if none is passed
        optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)

    train_losses = []
    val_losses = []
    test_losses = []

    train_pccs = []
    val_pccs = []
    test_pccs = []

    train_rmses = []
    val_rmses = []
    test_rmses = []

    train_pccs_channelwise = []
    val_pccs_channelwise = []
    test_pccs_channelwise = []

    train_rmses_channelwise = []
    val_rmses_channelwise = []
    test_rmses_channelwise = []

    # Check for existing checkpoint to resume training
    last_epoch = 0
    checkpoint_path = f"/content/MyDrive/MyDrive/models/{filename}/"

    if train_from_last_epoch and os.path.exists(checkpoint_path):
        # Scan for the latest saved checkpoint
        checkpoints = [f for f in os.listdir(checkpoint_path) if f.endswith('.pth')]
        if checkpoints:
            checkpoints.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Sort by epoch number
            latest_checkpoint = checkpoints[-1]
            print(f"Loading model from checkpoint: {latest_checkpoint}")
            checkpoint = torch.load(os.path.join(checkpoint_path, latest_checkpoint))
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            last_epoch = checkpoint['epoch']  # Continue from the next epoch

            # Load the history from checkpoint
            train_losses = checkpoint['history']['train_losses']
            val_losses = checkpoint['history']['val_losses']
            test_losses = checkpoint['history']['test_losses']
            train_pccs = checkpoint['history']['train_pccs']
            val_pccs = checkpoint['history']['val_pccs']
            test_pccs = checkpoint['history']['test_pccs']
            train_rmses = checkpoint['history']['train_rmses']
            val_rmses = checkpoint['history']['val_rmses']
            test_rmses = checkpoint['history']['test_rmses']
            train_pccs_channelwise = checkpoint['history']['train_pccs_channelwise']
            val_pccs_channelwise = checkpoint['history']['val_pccs_channelwise']
            test_pccs_channelwise = checkpoint['history']['test_pccs_channelwise']
            train_rmses_channelwise = checkpoint['history']['train_rmses_channelwise']
            val_rmses_channelwise = checkpoint['history']['val_rmses_channelwise']
            test_rmses_channelwise = checkpoint['history']['test_rmses_channelwise']
            if 'curriculum_schedule' in checkpoint:
                curriculum_loader.curriculum_schedule = checkpoint['curriculum_schedule']  # Load saved curriculum schedule
        else:
            print("No checkpoints found, starting from scratch.")
    else:
        print("Starting from scratch.")

    start_time = time.time()
    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    for epoch in range(last_epoch, epochs):
        epoch_start_time = time.time()
        model.train()

        if curriculum_loader:
            curriculum_loader.update_epoch(epoch)
            train_loader, val_loader, test_loader = curriculum_loader.get_loaders()

        # Track total loss
        epoch_train_total_loss = 0.0
        epoch_train_regression_loss = 0.0
        epoch_train_classification_loss = 0.0

        epoch_train_pcc = np.zeros(len(config.channels_joints))
        epoch_train_rmse = np.zeros(len(config.channels_joints))

        for i, (data_acc, data_gyr, target, data_EMG, subject_id) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} Training")):
            optimizer.zero_grad()

            #print dimensions of each and subject id
            # print("acc",data_acc.shape)
            # print("gyro",data_gyr.shape)
            # print("target",target.shape)
            # print("emg",data_EMG.shape)
            # print("subject ids",subject_id)

            # Move data to device
            data_acc = data_acc.to(device).float()
            data_gyr = data_gyr.to(device).float()
            data_EMG = data_EMG.to(device).float()
            target = target.to(device).float()
            subject_id = subject_id.to(device).long() - 1  # Adjust subject IDs to start from 0

            # Forward pass
            output, out_classifier, _ = model(data_acc, data_gyr, data_EMG)

            # print(f"Output shape: {output.shape}")
            # print(f"Target shape: {target.shape}")
            # Compute regression loss
            loss_regression = criterion_regression(output, target)

            # Compute classification loss
            # Ensure shapes are correct
            # print(f"Out Classifier shape: {out_classifier.shape}")
            # print(f"Subject ID shape: {subject_id.shape}")
            # print(f"Subject ID dtype: {subject_id.dtype}")
            # print(f"Out Classifier dtype: {out_classifier.dtype}")
            # print(f"subject ids",subject_id)
            # print(f"out classifier",out_classifier)
            loss_classification = criterion_classification(out_classifier, subject_id)

            # Combine losses
            total_loss = loss_regression + alpha * loss_classification

            # Apply L1 regularization if specified
            if l1_lambda is not None:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                total_loss += l1_lambda * l1_norm

            # Backward pass and optimization
            total_loss.backward()
            optimizer.step()



            # Detach tensors and move to CPU to prevent issues with gradient computation
            batch_rmse, batch_pcc, _, _, _ = RMSE_prediction(
                output.detach().cpu().numpy(),
                target.detach().cpu().numpy(),
                len(config.channels_joints),
                print_losses=False
            )

            # Update metrics
            epoch_train_total_loss += total_loss.item()
            epoch_train_regression_loss += loss_regression.item()
            epoch_train_classification_loss += loss_classification.item()

            epoch_train_pcc += batch_pcc
            epoch_train_rmse += batch_rmse

        # Calculate average losses
        avg_train_total_loss = epoch_train_total_loss / len(train_loader)
        avg_train_regression_loss = epoch_train_regression_loss / len(train_loader)
        avg_train_classification_loss = epoch_train_classification_loss / len(train_loader)

        avg_train_pcc = epoch_train_pcc / len(train_loader)
        avg_train_rmse = epoch_train_rmse / len(train_loader)

        train_losses.append(avg_train_total_loss)
        train_pccs.append(np.mean(avg_train_pcc))  # Overall average PCC
        train_rmses.append(np.mean(avg_train_rmse))  # Overall average RMSE

        # Save channel-wise metrics
        train_pccs_channelwise.append(avg_train_pcc)  # Per channel
        train_rmses_channelwise.append(avg_train_rmse)  # Per channel

        # Evaluate on validation set
        avg_val_total_loss, avg_val_pcc, avg_val_rmse = evaluate_model(
            device,
            model,
            val_loader,
            criterion_regression
        )

        val_losses.append(avg_val_total_loss)
        val_pccs.append(np.mean(avg_val_pcc))  # Overall average PCC
        val_rmses.append(np.mean(avg_val_rmse))  # Overall average RMSE

        # Save channel-wise metrics
        val_pccs_channelwise.append(avg_val_pcc)  # Per channel
        val_rmses_channelwise.append(avg_val_rmse)  # Per channel

        # Evaluate on test set
        avg_test_total_loss, avg_test_pcc, avg_test_rmse = evaluate_model(
            device,
            model,
            test_loader,
            criterion_regression
        )

        test_losses.append(avg_test_total_loss)
        test_pccs.append(np.mean(avg_test_pcc))  # Overall average PCC
        test_rmses.append(np.mean(avg_test_rmse))  # Overall average RMSE

        # Save channel-wise metrics
        test_pccs_channelwise.append(avg_test_pcc)  # Per channel
        test_rmses_channelwise.append(avg_test_rmse)  # Per channel

        print(f"Epoch: {epoch + 1}, Training Total Loss: {avg_train_total_loss:.4f}, Validation Total Loss: {avg_val_total_loss:.4f}, Test Total Loss: {avg_test_total_loss:.4f}")
        print(f"Training Regression Loss: {avg_train_regression_loss:.4f}, Validation Regression Loss: {avg_val_total_loss:.4f}, Test Regression Loss: {avg_test_total_loss:.4f}")
        print(f"Training Classification Loss: {avg_train_classification_loss:.4f}, Validation Classification Loss: {avg_val_total_loss:.4f}, Test Classification Loss: {avg_test_total_loss:.4f}")
        print(f"Training RMSE: {np.mean(avg_train_rmse):.4f}, Validation RMSE: {np.mean(avg_val_rmse):.4f}, Test RMSE: {np.mean(avg_test_rmse):.4f}")
        print(f"Training PCC: {np.mean(avg_train_pcc):.4f}, Validation PCC: {np.mean(avg_val_pcc):.4f}, Test PCC: {np.mean(avg_test_pcc):.4f}")

        # Save checkpoint, including curriculum schedule
        if not os.path.exists(checkpoint_path):
            os.makedirs(checkpoint_path)

        # Save checkpoint with the curriculum schedule
        history = {
            'train_losses': train_losses,
            'val_losses': val_losses,
            'test_losses': test_losses,
            'train_pccs': train_pccs,
            'val_pccs': val_pccs,
            'test_pccs': test_pccs,
            'train_rmses': train_rmses,
            'val_rmses': val_rmses,
            'test_rmses': test_rmses,
            'train_pccs_channelwise': train_pccs_channelwise,
            'val_pccs_channelwise': val_pccs_channelwise,
            'test_pccs_channelwise': test_pccs_channelwise,
            'train_rmses_channelwise': train_rmses_channelwise,
            'val_rmses_channelwise': val_rmses_channelwise,
            'test_rmses_channelwise': test_rmses_channelwise
        }

        save_checkpoint(
            model,
            optimizer,
            epoch,
            f"{checkpoint_path}/{filename}_epoch_{epoch + 1}.pth",
            train_loss=avg_train_total_loss,
            val_loss=avg_val_total_loss,
            test_loss=avg_test_total_loss,
            channelwise_metrics={
                'train': {'pcc': avg_train_pcc, 'rmse': avg_train_rmse},
                'val': {'pcc': avg_val_pcc, 'rmse': avg_val_rmse},
                'test': {'pcc': avg_test_pcc, 'rmse': avg_test_rmse},
            },
            history=history,  # Save history in the checkpoint
            curriculum_schedule=curriculum_loader.curriculum_schedule if curriculum_loader else None  # Save curriculum schedule
        )

        # Early stopping logic
        if avg_val_total_loss < best_val_loss:
            best_val_loss = avg_val_total_loss
            torch.save(model.state_dict(), filename)
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Stopping early after {epoch + 1} epochs")
            break

    end_time = time.time()
    print(f"Total training time: {end_time - start_time:.2f} seconds")

    return model, train_losses, val_losses, test_losses, train_pccs, val_pccs, test_pccs, train_rmses, val_rmses, test_rmses







In [21]:
# @title Helper Functions


# Function to create the teacher model with defaults from config
def create_teacher_model(input_acc, input_gyr, input_emg, base_weights_path=None, drop_prob=0.25, w=100,num_subjects=None):
    model = teacher(input_acc, input_gyr, input_emg, drop_prob=drop_prob, w=w,num_subjects=num_subjects)

    if base_weights_path:
        # Load the initial weights from the base model
        model.load_state_dict(torch.load(base_weights_path))

    return model


In [22]:

import os
import h5py
import csv
from tqdm.notebook import tqdm
import pandas as pd
def create_curriculum_schedule(all_subjects, num_subjects, epochs):
    return [(epoch, sorted(random.sample(all_subjects, num_subjects))) for epoch in range(epochs)]

# Define the list of all subjects
all_subjects = ['subject_1','subject_2', 'subject_3', 'subject_4', 'subject_5', 'subject_6',
                'subject_7', 'subject_8', 'subject_9', 'subject_10',
                'subject_11', 'subject_12', 'subject_13']

# Define the base model and save its weights
input_acc, input_gyr, input_emg = 18, 18, 3  # Example inputs based on your setup
base_model = teacher(input_acc, input_gyr, input_emg)
base_weights_path = 'base_teacher_weights.pth'
torch.save(base_model.state_dict(), base_weights_path)
window_overlap = 0
batch_size = 64
curriculum_epochs = 50

# Create model configurations for each subject as the test subject
model_configs = {}

test_subject = 'subject_1'
train_subjects = [subject for subject in all_subjects if subject not in test_subject]

# Define the number of subjects/classes
num_classes = 12

# List of different weights for the classification loss
alpha_values = [.2,.4,.8,1,2,4,8]


for alpha in alpha_values:
    model_name = f'TeacherModel_DomainInvariant_alpha_{alpha}_wl{100}_ol{75}'
    model_configs[model_name] = {
        'model': create_teacher_model(
            input_acc=input_acc,
            input_gyr=input_gyr,
            input_emg=input_emg,
            w=100,
            num_subjects=num_classes  # Ensure the model includes the classification head
        ),
        'loss': RMSELoss(),
        'loaders': create_base_data_loaders(
            config=config,
            train_subjects=train_subjects,
            test_subjects=[test_subject],
            window_length=100,
            window_overlap=75,
            batch_size=batch_size
        ),
        'epochs': curriculum_epochs,
        'use_curriculum': False,
        'alpha': alpha,  # Weighting factor for classification loss
        'num_classes': num_classes,  # Number of classes for the classification task
    }

# Output the model configurations for debugging (optional)
for model_name, model_conf in model_configs.items():
    print(f"Model: {model_name}")


Sharded data found at /content/datasets/dataset_wl100_ol75_train_2_3_4_5_6_7_8_9_10_11_12_13. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol0_test_1. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol75_train_2_3_4_5_6_7_8_9_10_11_12_13. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol0_test_1. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol75_train_2_3_4_5_6_7_8_9_10_11_12_13. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol0_test_1. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol75_train_2_3_4_5_6_7_8_9_10_11_12_13. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol0_test_1. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol75_train_2_3_4_5_6_7_8_9_10_11_12_13. Skipping resharding.
Sharded data found at /content/datasets/dataset_wl100_ol0_test_1. Skipping re

In [23]:
 # @title run models

#clear gpu memory
torch.cuda.empty_cache()

def ask_run():
    response = input("Do you want to run models? (yes/no): ").strip().lower()
    if response in ['yes', 'y']:
        return True
    elif response in ['no', 'n']:
        return False
    else:
        print("Invalid input. Please enter 'yes' or 'no'.")
        return ask_run()  # Recursively ask again until valid input is given

run = ask_run()

if run:
    for model_name, model_config in model_configs.items():
        model = model_config['model']
        loss_function = model_config['loss']  # May be None, as loss functions are defined within train_teacher
        epochs = model_config.get("epochs", 100)
        device = model_config.get("device", torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        learn_rate = model_config.get("learn_rate", 0.001)
        use_curriculum = model_config.get("use_curriculum", False)

        optimizer = model_config.get("optimizer", None)
        l1_lambda = model_config.get("l1_lambda", None)
        alpha = model_config.get('alpha', 1.0)
        num_classes = model_config.get('num_classes', 12)  # Default to 12 if not specified

        print(f"Running model: {model_name} with alpha: {alpha}")

        if use_curriculum:
            curriculum_loader = model_config['loader']  # Get the CurriculumDataLoader
            train_loader, val_loader, test_loader = None, None, None  # Curriculum will handle loading per epoch

            model, train_losses, val_losses, test_losses, train_pccs, val_pccs, test_pccs, \
            train_rmses, val_rmses, test_rmses = train_teacher(
                device=device,
                train_loader=train_loader,  # Placeholders; curriculum will manage loaders dynamically
                val_loader=val_loader,
                test_loader=test_loader,
                learn_rate=learn_rate,
                epochs=epochs,
                model=model,
                filename=model_name,
                loss_function=loss_function,
                curriculum_loader=curriculum_loader,  # Pass the curriculum loader here
                optimizer=optimizer,
                l1_lambda=l1_lambda,
                train_from_last_epoch=model_config.get("train_from_last_epoch", False),
                alpha=alpha,
                num_classes=num_classes,
            )
        else:
            # Unpack the static loaders tuple (train_loader, val_loader, test_loader)
            train_loader, val_loader, test_loader = model_config['loaders']

            model, train_losses, val_losses, test_losses, train_pccs, val_pccs, test_pccs, \
            train_rmses, val_rmses, test_rmses = train_teacher(
                device=device,
                train_loader=train_loader,
                val_loader=val_loader,
                test_loader=test_loader,
                learn_rate=learn_rate,
                epochs=epochs,
                model=model,
                filename=model_name,
                loss_function=loss_function,
                optimizer=optimizer,
                l1_lambda=l1_lambda,
                train_from_last_epoch=model_config.get("train_from_last_epoch", False),
                alpha=alpha,
                num_classes=num_classes,
            )

        print(f"Finished training for {model_name}.")


Do you want to run models? (yes/no): yes
Running model: TeacherModel_DomainInvariant_alpha_0.2_wl100_ol75 with alpha: 0.2
Starting from scratch.


Epoch 1/50 Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch: 1, Training Total Loss: 19.3082, Validation Total Loss: 11.3202, Test Total Loss: 22.5309
Training Regression Loss: 18.8219, Validation Regression Loss: 11.3202, Test Regression Loss: 22.5309
Training Classification Loss: 2.4314, Validation Classification Loss: 11.3202, Test Classification Loss: 22.5309
Training RMSE: 18.3245, Validation RMSE: 10.9216, Test RMSE: 21.3135
Training PCC: 0.8002, Validation PCC: 0.9467, Test PCC: 0.6469
Checkpoint saved for epoch 1


Epoch 2/50 Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch: 2, Training Total Loss: 11.2280, Validation Total Loss: 9.6751, Test Total Loss: 20.6907
Training Regression Loss: 10.7971, Validation Regression Loss: 9.6751, Test Regression Loss: 20.6907
Training Classification Loss: 2.1549, Validation Classification Loss: 9.6751, Test Classification Loss: 20.6907
Training RMSE: 10.3655, Validation RMSE: 9.0129, Test RMSE: 19.3641
Training PCC: 0.9529, Validation PCC: 0.9647, Test PCC: 0.6568
Checkpoint saved for epoch 2


...

Epoch: 9, Training Total Loss: 358775.9338, Validation Total Loss: 3192.0979, Test Total Loss: 3196.6396
Training Regression Loss: 3553.0644, Validation Regression Loss: 3192.0979, Test Regression Loss: 3196.6396
Training Classification Loss: 44402.8587, Validation Classification Loss: 3192.0979, Test Classification Loss: 3196.6396
Training RMSE: 2411.4980, Validation RMSE: 1983.6934, Test RMSE: 1988.9080
Training PCC: 0.0007, Validation PCC: -0.0076, Test PCC: 0.0247
Checkpoint saved for epoch 9


Epoch 10/50 Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch: 10, Training Total Loss: 426810.1327, Validation Total Loss: 3667.6543, Test Total Loss: 3667.8897
Training Regression Loss: 4396.0631, Validation Regression Loss: 3667.6543, Test Regression Loss: 3667.8897
Training Classification Loss: 52801.7587, Validation Classification Loss: 3667.6543, Test Classification Loss: 3667.8897
Training RMSE: 3084.1037, Validation RMSE: 2261.4525, Test RMSE: 2260.3864
Training PCC: -0.0029, Validation PCC: 0.0220, Test PCC: 0.0110
Checkpoint saved for epoch 10


Epoch 11/50 Training:   0%|          | 0/327 [00:00<?, ?it/s]

Epoch: 11, Training Total Loss: 444482.0074, Validation Total Loss: 3297.5413, Test Total Loss: 3292.2595
Training Regression Loss: 4909.7258, Validation Regression Loss: 3297.5413, Test Regression Loss: 3292.2595
Training Classification Loss: 54946.5351, Validation Classification Loss: 3297.5413, Test Classification Loss: 3292.2595
Training RMSE: 3342.0212, Validation RMSE: 2362.8825, Test RMSE: 2361.7018
Training PCC: 0.0069, Validation PCC: 0.0047, Test PCC: 0.0291
Checkpoint saved for epoch 11
Stopping early after 11 epochs
Total training time: 1661.31 seconds
Finished training for TeacherModel_DomainInvariant_alpha_8_wl100_ol75.


In [24]:
# @title Plotting
import matplotlib.pyplot as plt
def load_most_recent_checkpoint(folder):
    """
    Loads the most recent checkpoint from the given folder and returns it.
    """
    checkpoint_files = [f for f in os.listdir(folder) if f.endswith('.pth')]
    if not checkpoint_files:
        print(f"No checkpoints found in {folder}")
        return None

    # Sort the checkpoint files by epoch (assuming filenames include the epoch number)
    checkpoint_files.sort(key=lambda f: int(f.split('_epoch_')[-1].split('.')[0]))

    # Load the most recent checkpoint
    latest_checkpoint = checkpoint_files[-1]
    checkpoint_path = os.path.join(folder, latest_checkpoint)
    checkpoint = torch.load(checkpoint_path)

    print(f"Loaded checkpoint: {latest_checkpoint} from {folder}")

    # Return the loaded checkpoint, which contains model state, optimizer state, and history
    return checkpoint

def plot_metrics_subplots(aggregated_metrics, folder):
    """Plots train/val/test losses, PCCs, and RMSEs in subplots."""
    epochs = list(range(1, len(aggregated_metrics['train_losses']) + 1))
    line_styles = ['-', '--', '-.', ':']  # Define different line styles for channels

    # New color palette
    train_color = '#17becf'  # Teal blue for Train
    val_color = '#bcbd22'    # Mustard yellow for Val
    test_color = '#e377c2'   # Pastel magenta for Test

    # Create subplots for losses, PCCs, and RMSEs
    fig, axes = plt.subplots(3, 2, figsize=(12, 14))
    fig.suptitle(f'Metrics over Epochs for {folder}', fontsize=16)

    # Plot Losses (subplot 1, 1)
    axes[0, 0].plot(epochs, aggregated_metrics['train_losses'], label='Train Loss', color=train_color, linestyle=line_styles[0])
    axes[0, 0].plot(epochs, aggregated_metrics['val_losses'], label='Val Loss', color=val_color, linestyle=line_styles[0])
    axes[0, 0].plot(epochs, aggregated_metrics['test_losses'], label='Test Loss', color=test_color, linestyle=line_styles[0])
    axes[0, 0].set_title('Losses over Epochs')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()

    # Plot Average PCCs (already averaged, so no need to take mean again) (subplot 1, 2)
    axes[0, 1].plot(epochs, aggregated_metrics['train_pccs'], label='Avg Train PCC', color=train_color, linestyle='-')
    axes[0, 1].plot(epochs, aggregated_metrics['val_pccs'], label='Avg Val PCC', color=val_color, linestyle='-')
    axes[0, 1].plot(epochs, aggregated_metrics['test_pccs'], label='Avg Test PCC', color=test_color, linestyle='-')
    axes[0, 1].set_title('Average PCC over Epochs')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('PCC')
    axes[0, 1].legend()

    # Plot PCCs for each channel (subplot 2, 1)
    for i, style in enumerate(line_styles):
        if len(aggregated_metrics['train_pccs_channelwise'][0]) > i:  # Ensure enough channels exist
            axes[1, 0].plot(epochs, [pcc[i] for pcc in aggregated_metrics['train_pccs_channelwise']], label=f'Train PCC (Ch {i+1})', color=train_color, linestyle=style)
            axes[1, 0].plot(epochs, [pcc[i] for pcc in aggregated_metrics['val_pccs_channelwise']], label=f'Val PCC (Ch {i+1})', color=val_color, linestyle=style)
            axes[1, 0].plot(epochs, [pcc[i] for pcc in aggregated_metrics['test_pccs_channelwise']], label=f'Test PCC (Ch {i+1})', color=test_color, linestyle=style)
    axes[1, 0].set_title('PCCs over Epochs (Per Channel)')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('PCC')
    axes[1, 0].legend()

    # Plot Average RMSEs (already averaged, so no need to take mean again) (subplot 2, 2)
    axes[1, 1].plot(epochs, aggregated_metrics['train_rmses'], label='Avg Train RMSE', color=train_color, linestyle='-')
    axes[1, 1].plot(epochs, aggregated_metrics['val_rmses'], label='Avg Val RMSE', color=val_color, linestyle='-')
    axes[1, 1].plot(epochs, aggregated_metrics['test_rmses'], label='Avg Test RMSE', color=test_color, linestyle='-')
    axes[1, 1].set_title('Average RMSE over Epochs')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('RMSE')
    axes[1, 1].legend()

    # Plot RMSEs for each channel (subplot 3, 1)
    for i, style in enumerate(line_styles):
        if len(aggregated_metrics['train_rmses_channelwise'][0]) > i:
            axes[2, 0].plot(epochs, [rmse[i] for rmse in aggregated_metrics['train_rmses_channelwise']], label=f'Train RMSE (Ch {i+1})', color=train_color, linestyle=style)
            axes[2, 0].plot(epochs, [rmse[i] for rmse in aggregated_metrics['val_rmses_channelwise']], label=f'Val RMSE (Ch {i+1})', color=val_color, linestyle=style)
            axes[2, 0].plot(epochs, [rmse[i] for rmse in aggregated_metrics['test_rmses_channelwise']], label=f'Test RMSE (Ch {i+1})', color=test_color, linestyle=style)
    axes[2, 0].set_title('RMSEs over Epochs (Per Channel)')
    axes[2, 0].set_xlabel('Epoch')
    axes[2, 0].set_ylabel('RMSE')
    axes[2, 0].legend()

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the suptitle
    plt.show()

def plot_comparison_bar_graphs_with_limits(aggregated_metrics_dict):
    """
    Plots two bar graphs comparing the best RMSE and PCC for each model, showing corresponding PCC/RMSE from the same epoch,
    and plots average test RMSEs across epochs for each model.
    """

    # Dictionary to hold the best RMSE and PCC along with their associated metrics at the same epoch
    best_metrics = {}

    # Iterate through the aggregated metrics and extract the best RMSE and its corresponding PCC, and vice versa
    for model_name, metrics in aggregated_metrics_dict.items():
        # Extract pre-averaged RMSEs and PCCs for the test data
        test_rmses = metrics['test_rmses']  # Pre-averaged RMSE values
        test_pccs = metrics['test_pccs']    # Pre-averaged PCC values
        val_losses = metrics['val_losses']  # Pre-averaged validation losses

        # Find the epoch index for the best RMSE and the corresponding PCC at that epoch
        best_rmse_epoch = np.argmin(val_losses)
        best_rmse = test_rmses[best_rmse_epoch]
        corresponding_pcc_for_best_rmse = test_pccs[best_rmse_epoch]

        # Find the epoch index for the best PCC and the corresponding RMSE at that epoch
        best_pcc_epoch = np.argmax(test_pccs)
        best_pcc = test_pccs[best_pcc_epoch]
        corresponding_rmse_for_best_pcc = test_rmses[best_pcc_epoch]

        # Store in a dictionary for plotting
        best_metrics[model_name] = {
            'best_rmse': best_rmse,
            'corresponding_pcc_for_best_rmse': corresponding_pcc_for_best_rmse,
            'best_pcc': best_pcc,
            'corresponding_rmse_for_best_pcc': corresponding_rmse_for_best_pcc
        }

    # Convert to lists for sorting
    models = list(best_metrics.keys())
    best_rmses = [best_metrics[model]['best_rmse'] for model in models]
    corresponding_pccs = [best_metrics[model]['corresponding_pcc_for_best_rmse'] for model in models]
    best_pccs = [best_metrics[model]['best_pcc'] for model in models]
    corresponding_rmses = [best_metrics[model]['corresponding_rmse_for_best_pcc'] for model in models]

    # Sort by RMSE and PCC
    sorted_by_rmse = sorted(zip(models, best_rmses, corresponding_pccs), key=lambda x: x[1], reverse=False)
    sorted_by_pcc = sorted(zip(models, best_pccs, corresponding_rmses), key=lambda x: x[1], reverse=True)

    # Unpack sorted values
    sorted_models_rmse, sorted_rmses, sorted_pccs_for_rmse = zip(*sorted_by_rmse)
    sorted_models_pcc, sorted_pccs, sorted_rmses_for_pcc = zip(*sorted_by_pcc)

    # Ensure consistent colors across both graphs
    colors = sns.color_palette("husl", len(models))
    model_colors_rmse = {model: colors[i] for i, model in enumerate(sorted_models_rmse)}
    model_colors_pcc = {model: model_colors_rmse[model] for model in sorted_models_pcc}  # Keep the colors consistent

    # Plot RMSE Bar Graph (With Corresponding PCC values)
    plt.figure(figsize=(10, 6))
    bars_rmse = plt.bar(sorted_models_rmse, sorted_rmses, color=[model_colors_rmse[model] for model in sorted_models_rmse])
    plt.title('Best RMSE per Model (Sorted)')
    plt.xlabel('Model')
    plt.ylabel('Best RMSE')
    plt.ylim(0, 25)  # Set an appropriate y-limit for RMSEs
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Add the actual RMSE values and corresponding PCC on top of the bars
    for i, bar in enumerate(bars_rmse):
        yval_rmse = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval_rmse + 0.5, f'{yval_rmse:.2f}', ha='center', va='bottom')
        plt.text(bar.get_x() + bar.get_width()/2, yval_rmse - 2, f'PCC: {sorted_pccs_for_rmse[i]:.2f}', ha='center', va='top', color='black')

    plt.show()

    # Plot PCC Bar Graph (With Corresponding RMSE values)
    plt.figure(figsize=(10, 6))
    bars_pcc = plt.bar(sorted_models_pcc, sorted_pccs, color=[model_colors_pcc[model] for model in sorted_models_pcc])
    plt.title('Best PCC per Model (Sorted)')
    plt.xlabel('Model')
    plt.ylabel('Best PCC')
    plt.ylim(0, 1)  # Set an appropriate y-limit for PCCs
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Add the actual PCC values and corresponding RMSE on top of the bars
    for i, bar in enumerate(bars_pcc):
        yval_pcc = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval_pcc + 0.02, f'{yval_pcc:.2f}', ha='center', va='bottom')
        plt.text(bar.get_x() + bar.get_width()/2, yval_pcc - 0.08, f'RMSE: {sorted_rmses_for_pcc[i]:.2f}', ha='center', va='top', color='black')

    plt.show()

    # Plot the average test RMSEs across epochs for each model
    plt.figure(figsize=(10, 6))
    for i, model in enumerate(models):
        avg_test_rmses = aggregated_metrics_dict[model]['test_rmses']  # Directly plot pre-averaged RMSEs
        plt.plot(avg_test_rmses, label=f'{model} Average Test RMSE', color=colors[i])

    plt.title('Average Test RMSE per Epoch for All Models')
    plt.xlabel('Epoch')
    plt.ylabel('Average RMSE')
    plt.legend()
    plt.tight_layout()
    plt.show()


# Initialize an empty dictionary to hold metrics for all models
aggregated_metrics_dict = {}

# Loop over each model and load the checkpoint
for model_name, model_config in model_configs.items():
    # Construct the folder path based on the filename
    folder = f"/content/MyDrive/MyDrive/models/{model_name}"
    checkpoint = load_most_recent_checkpoint(folder)

    if checkpoint:
        aggregated_metrics = checkpoint['history']

        # Store the metrics for comparison later
        aggregated_metrics_dict[model_name] = {
            'test_rmses': aggregated_metrics['test_rmses'],
            'test_pccs': aggregated_metrics['test_pccs'],
            'train_losses': aggregated_metrics['train_losses'],
            'val_losses': aggregated_metrics['val_losses'],
        }

        # Plot metrics for this model
        plot_metrics_subplots(aggregated_metrics, model_name)

# After collecting all model metrics, plot the comparison bar graphs
if aggregated_metrics_dict:
    plot_comparison_bar_graphs_with_limits(aggregated_metrics_dict)


Output hidden; open in https://colab.research.google.com to view.